In [21]:
from fuzzywuzzy import fuzz

list_categories = ['phone', 'monitor']
list_brands = ['apple', 'lg', 'microsoft']
list_features = ['wired', '4k', '20in', 'pro']

def identify_matching_value_sale_label(label, list_, threshold=90):
    list_similarity = [
        (txt, fuzz.ratio(txt.lower(), sub_label.lower())) 
        for txt in list_ 
        for sub_label in label.split(' ')
    ]
    
    if max(list_similarity, key=lambda x: x[1])[1] > threshold:
        return max(list_similarity, key=lambda x: x[1])[0]
    else:
        return ''

def match_sale_to_product_sheet(sale_label):
    """
    Input : label of the sale 
    Output : product sheet id linked to the sale 
    """
    category_identified, brand_identified, product_features_identified = "", "", []

    # 0. verification and pre-processing on the input 

    # 1. identify the category of the product (phone, monitor, ...)
    category_identified = identify_matching_value_sale_label(sale_label, list_categories, threshold=90)
    if category_identified == "":
        return False

    # 2. identify the brand of the product (Apple, LG, ...)
    # list_brands should be obtained through a query of brands linked to the category identified
    # the category identified should be removed from the sale_label 
    brand_identified = identify_matching_value_sale_label(sale_label, list_brands, threshold=90)

    # 3. identify the feature of the product (ultrawide, version 2, ...)

    # 4. find the product sheet matching those caracteristics 

    print('Input : ', sale_label)
    print('. Category identified : ', category_identified)
    print('. Brand identified : ', brand_identified)

match_sale_to_product_sheet('Apple Iphone pro 14')


Input :  Apple Iphone pro 14
. Category identified :  phone
. Brand identified :  apple
